In [ ]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy


import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F


from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

### Base GRU model

In [ ]:
input_dim = 25
pad_token = np.zeros(input_dim)
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))

    return np.array(sents_padded)

In [ ]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, x_lab, x_demo, y):
        self.x_lab = x_lab
        self.x_demo = x_demo
        self.y = y

    def __getitem__(self, index): # 返回的是tensor
        return self.x_lab[index], self.x_demo[index], self.y[index]

    def __len__(self):
        return len(self.x_lab)

In [ ]:
data_path = './dataset/tongji/processed_data/'
file_name = './ckpt/gru.pth'
small_part = False
arg_timestep = 1.0
batch_size = 16
epochs = 100
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
#device = torch.device('cpu')
print("available device: {}".format(device))


x_lab = pickle.load(open('./dataset/tongji/processed_data/train_x_labtest_outcome.pkl', 'rb'))
x_lab = np.array(x_lab, dtype=object)

x_demo = pickle.load(open('./dataset/tongji/processed_data/train_x_demographic_outcome.pkl', 'rb'))
x_demo = np.array(x_demo)

y = pickle.load(open('./dataset/tongji/processed_data/train_y_outcome.pkl', 'rb'))
y = np.array(y)

# print(len(x_lab[0][0]))
x_lab = pad_sents(x_lab, pad_token)
# len(x_lab[7])

In [ ]:
train_dataset = Dataset(x_lab, x_demo, y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

print(x_lab.shape)
print(x_demo.shape)
print(y.shape)
# print(y)


In [ ]:
class GRU(nn.Module):
    def __init__(self, input_lab_dim, input_demo_dim, hidden_dim, output_dim, act_layer=nn.GELU, drop=0.):
        super(GRU, self).__init__()

        # hyperparameters
        self.input_lab_dim = input_lab_dim
        self.input_demo_dim = input_demo_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.demo_proj = nn.Linear(input_demo_dim, hidden_dim)
        self.lab_proj = nn.Linear(input_lab_dim, hidden_dim)

        self.gru = nn.GRU(input_size = hidden_dim, hidden_size = hidden_dim, num_layers = 1, batch_first = True)
        
        self.act = act_layer()
        self.fc = nn.Linear(2*hidden_dim, output_dim)
        self.drop = nn.Dropout(drop)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x_lab, x_demo):

        x_lab = self.lab_proj(x_lab)
        x_lab = self.act(x_lab) 

        _, x_lab = self.gru(x_lab) # (1, batch_size, hidden_dim)
        x_lab = x_lab[0] # (batch_size, hidden_dim)

        x_demo = self.demo_proj(x_demo)
        x_demo = self.act(x_demo) # (batch_size, hidden_dim)

        x = torch.cat((x_lab, x_demo), 1) # (batch_size, 2*hidden_dim)

        x = self.drop(x)
        x = self.fc(x)
        x = self.drop(x)
        
        x = self.sigmoid(x)
        return x


In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) # numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) # gpu
torch.backends.cudnn.deterministic=True # cudnn
np.set_printoptions(threshold=np.inf, precision=2, suppress=True)


model = GRU(input_lab_dim=25, input_demo_dim=2, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for each_epoch in range(epochs):
    batch_loss = []
    model.train()
    for step, data in enumerate(train_loader):   
        batch_x_lab, batch_x_demo, batch_y = data
        batch_x_lab, batch_x_demo, batch_y = batch_x_lab.float(), batch_x_demo.float(), batch_y.float()
        batch_y = batch_y.unsqueeze(-1)

        optimizer.zero_grad()
        output = model(batch_x_lab, batch_x_demo)
        # print(output)
        loss = get_loss(output, batch_y)
        batch_loss.append(loss.item())

        # print(output.shape, batch_y.shape, loss)
        loss.backward()
        optimizer.step()
        
        if step % 10 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, torch.mean(torch.Tensor(batch_loss))))
 